In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas_bokeh as pb
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, export_png, output_file
from bokeh.layouts import row, gridplot
from bokeh.models import ColumnDataSource
import numpy as np
import random
# import seaborn as sns; sns.set()
# %matplotlib inline
# pb.output_notebook()
# output_notebook()

In [2]:
# 已知规则
# 1. 客户来源_原始 如果该字段为空，则自动填充为 展厅

In [3]:
basedir = "/home/orin/Learning/Data"

In [4]:
dFilename = "2020年10月潜客数据.xlsx"
cFilename = "车型代码20201111_F.xlsx"

In [4]:
# databook = pd.read_excel("%s/%s" % (basedir, dFilename), sheet_name=0)

In [2]:
# databook.to_pickle("potential.pkl")
databook = pd.read_pickle("potential.pkl")

In [3]:
# 图表所需的层次
graphLVL = ["大区名称", "小区名称", "城市", "经销商代码"]

In [119]:
def transArr(df):
    tmp = []
    for idx, row in df.iterrows():
        tmp.extend([row['车型new']] * row['潜客量'])
    return tmp

In [121]:
transArr(dataCOMPUTed[(dataCOMPUTed['经销商代码']=='SA22015')&(dataCOMPUTed['客户来源更新']=='展厅')&(dataCOMPUTed['潜客量']>0)][['车型new', '潜客量']])

['A4L',
 'A4L',
 'A4L',
 'A4L',
 'A4L',
 'A6L',
 'A6L',
 'A6L',
 'A6L',
 'A6L',
 'Q2L',
 'Q3',
 'Q5L',
 'Q5L',
 'Q5L',
 'Q5L',
 'Q5L',
 'others']

In [107]:
def genNewData(dCode, cSource):
    tmpArr = transArr(dataCOMPUTed[(dataCOMPUTed['经销商代码']==dCode)&(dataCOMPUTed['客户来源更新']==cSource)&(dataCOMPUTed['潜客量']>0)][['车型new', '潜客量']])
    while tmpArr:
        

In [4]:
def tempDF(lvl, df):
    tmp = df[[lvl, '客户来源更新', '车型new', '潜客量']].copy()
    tmp = tmp.groupby([lvl, '客户来源更新', '车型new'])['潜客量'].sum().reset_index()
    tmp['%s汇总' % lvl] = tmp.groupby([lvl, '客户来源更新'])['潜客量'].transform('sum')
    tmp['车型占比'] = tmp['潜客量']/tmp['%s汇总' % lvl]
    tmp.drop(['潜客量', '%s汇总' % lvl], axis=1, inplace=True)
    return tmp

In [6]:
def batDraw(lvl):
    graphics = []
    vals = list(getIDX(lvl, dataUSEstd))
    df_o = tempDF(lvl, dataUSEstd)
    df_c = tempDF(lvl, dataEsti)

    for t in ["展厅", "线上"]:
        tmp = []
        for i in vals:
            tmp.append(drawGraph(
                df_o[(df_o[lvl]==i)&(df_o['客户来源更新']==t)][['车型new', '车型占比']],
                df_c[(df_c[lvl]==i)&(df_c['客户来源更新']==t)][['车型new', '车型占比']],
                '%s %s' % (i, t)
            ))
        graphics.append(tmp)

    return graphics

In [7]:
def drawGraph(se1, se2, title):
# def drawGraph(tmpdf, title):
    tmpdf = se1.merge(se2, on=['车型new'], suffixes=['_origin', '_fix'])
    m = tmpdf.plot_bokeh(kind='barh', legend='top_right', x='车型new', title=title, show_figure=False, zooming=False)
    return m

In [8]:
def drawGraph1(se1, se2, title):
# def drawGraph(tmpdf, title):
    p = figure(plot_width=200, plot_height=150, title=title)
    tmpdf = se1.merge(se2, on=['车型new'], suffixes=['_origin', '_fix'])
    print(tmpdf)
    # tmpdf.plot_bokeh(kind='line', legend='top_left', x='车型new', title=title)
    p.multi_line([tmpdf['车型new'], tmpdf['车型new']], [tmpdf.iloc[:, -2], tmpdf.iloc[:, -1]], show_figure=False)
    show(p)

In [9]:
def resetType(ser):
    if pd.isnull(ser['分组名称']):
        return np.nan
    else:
        return ser['车型']

In [10]:
def covTypes(df):
    for col in df.columns:
        if col == '潜客量':
            df[col] = df[col].astype('int32')
#         elif col== '日期':
#             df[col] = df[col].to_datetime()
        else:
            df[col] = df[col].astype('category')

In [11]:
def getIDX(cName, df, c=6):
    # 在df[cName]的最高30%里，随机取c个作为输出
    # 用于将来的分布验证
    tmp = df[[cName, '潜客量']].groupby([cName])['潜客量'].sum().reset_index()
    tmplist = df[cName].unique()
    if int(len(tmplist)*0.3) <= c:
        tmpidx = tmp.sort_values(by='潜客量', ascending=False).index.values
    else:
        tmpidx = tmp.sort_values(by='潜客量', ascending=False)[:int(len(tmplist)*0.3)].index.values
    return np.array(tmp[cName].iloc[random.sample(list(tmpidx), c)])

In [12]:
def balanceCom(rows):
    # 根据差值选择计算方式
    if rows['差值'] < 0:
        if rows['已进位'] and rows['顺序']+rows['差值']<=0:
            return rows['预计算潜客2'] - 1
    elif rows['差值'] > 0:
        if (not rows['已进位']) and rows['顺序'] - rows['差值'] <= 0:
            return rows['预计算潜客2'] + 1
    return rows['预计算潜客2']

In [13]:
def getColumns(col, vals,ty):
    # 获取画图的列
    oriData = dataUSEstd[(dataUSEstd[col]==vals)&(dataUSEstd['客户来源更新']==ty)][['车型new', '车型占比']]
    comData = dataEsti[(dataEsti[col]==vals)&(dataEsti['客户来源更新']==ty)][['车型new', '车型占比']]
    # oriData = statRES(col, dataUSEstd)[]
    return oriData.merge(comData, on='车型new', suffixes=['_origin', '_fix'])

In [115]:
getColumns('经销商代码', 'SA44031', '线上')

,车型new,车型占比_origin,车型占比_fix
0,A3,0.117647,0.119565
1,A4 FBU,0.019608,0.021739
2,A4L,0.156863,0.157609
3,A5,0.019608,0.021739
4,A6 FBU,0.019608,0.021739
5,A6L,0.274510,0.271739
6,A8L,0.019608,0.021739
7,Q2L,0.078431,0.076087
8,Q3,0.078431,0.076087
9,Q5L,0.078431,0.076087


In [14]:
covTypes(databook)

In [123]:
databook['车型new'] = databook.apply(resetType, axis=1)


In [133]:
dataAvailable = databook[databook['车型new'].notna()]
dataAvailable.drop(['分组名称', '客户来源_原始', '车型', '月份'], axis=1, inplace=True)

In [134]:
dataMiss = pd.DataFrame(columns=list(dataAvailable.columns))
dataMiss

,日期,大区名称,小区名称,经销商代码,经销商名称,城市,潜客量,大区,客户来源更新,车型new


In [132]:
dataAvailable.head(20)

,日期,大区名称,小区名称,经销商代码,经销商名称,城市,潜客量,大区,客户来源更新,车型new
0,2020-10-01,北部区,北京1区,SA11043,北京奥吉通国门,北京,1,A3,展厅,A4L
1,2020-10-01,北部区,北京1区,SA11043,北京奥吉通国门,北京,2,A3,展厅,A6L
2,2020-10-01,北部区,北京1区,SA11043,北京奥吉通国门,北京,1,A3,展厅,Q3
3,2020-10-01,北部区,北京1区,SA11043,北京奥吉通国门,北京,1,A3,展厅,Q5L
4,2020-10-01,北部区,北京1区,SA11043,北京奥吉通国门,北京,2,A3,线上,Q5L
5,2020-10-01,北部区,北京1区,SA11043,北京奥吉通国门,北京,1,A3,线上,others
8,2020-10-01,北部区,北京1区,SA11016,北京奥吉通,北京,1,A3,展厅,A4L
9,2020-10-01,北部区,北京1区,SA11016,北京奥吉通,北京,1,A3,展厅,Q3
10,2020-10-01,北部区,北京1区,SA11036,北京奥嘉世茂,北京,1,A3,线上,A4L
11,2020-10-01,北部区,北京1区,SA11036,北京奥嘉世茂,北京,1,A3,展厅,A4L


In [16]:
dataUSE = databook[['客户来源更新', '大区名称', '小区名称', '城市', '经销商代码', '车型new', '潜客量']].copy()

In [17]:
cityLIST = dataUSE['城市'].unique()
area1LIST = dataUSE['大区名称'].unique()
area2LIST = dataUSE['小区名称'].unique()
dealerLIST = dataUSE['经销商代码'].unique()
print(len(area1LIST), len(area2LIST), len(cityLIST), len(dealerLIST))

6 56 243 555


In [20]:
monthDays = len(databook['日期'].unique())

In [21]:
dataBYday = databook[databook['车型new'].isna()].groupby(['日期', '客户来源更新', '大区名称', '小区名称', '城市', '经销商代码'])['潜客量'].sum().reset_index()

In [15]:
# dealerDISTbd = databook[dataUSE['车型new'].notna()][["日期", "经销商代码", "客户来源更新", "车型new", "潜客量"]]

In [12]:
# 计算 经销商 客户来源 和车型 的group by sum，去掉日期来计算概率
# sum(经销商、客户来源、车型)/monthDays 这就是某经销商在某来源下车型出现的概率
# dealerDISTbd.groupby(["经销商代码", "客户来源更新", "车型new", "日期"])["潜客量"].sum().reset_index()
# 按日的做起来太麻烦所以不用这个了

In [22]:
dataUSEstd = dataUSE[dataUSE['车型new'].notna()]
dataUSE2b = dataUSE[dataUSE['车型new'].isna()]

In [23]:
# 汇总车型数据
dataUSEstd = dataUSEstd.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()

In [24]:
# 创建一个基于车型的城市分布
# 因为后续会有部分经销商没有对应 "客户来源更新" 的分布，比如某些经销商没有线上的已有数据作为基准，这时就用城市的替换
# dataCITYstd = dataUSEstd.groupby(['城市', '客户来源更新', '车型new'])['潜客量'].sum().reset_index()
dataCITYstd = dataUSEstd.groupby(['城市', '车型new'])['潜客量'].sum().reset_index()

In [25]:
# 可能存在来源更新没有的情况，处理起来太复杂，直接用城市了
# dataCITYstd['城市汇总'] = dataCITYstd.groupby(['城市', '客户来源更新'])['潜客量'].transform('sum')
dataCITYstd['城市汇总'] = dataCITYstd.groupby(['城市'])['潜客量'].transform('sum')
dataCITYstd['车型占比'] = dataCITYstd['潜客量']/ dataCITYstd['城市汇总']

In [26]:
dataUSEstd['dealer汇总'] = dataUSEstd.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新'])['潜客量'].transform('sum')
dataUSEstd['车型占比'] = dataUSEstd['潜客量']/dataUSEstd['dealer汇总']
dataUSEstd['日均概率'] = dataUSEstd['潜客量']/monthDays

In [27]:
dataUSE2b = dataUSE2b.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新'])['潜客量'].sum().reset_index()

In [28]:
dataUSEstd['潜客量'].sum()

314970

In [29]:
dataUSE2b['潜客量'].sum()

66503

In [30]:
datatmp = dataUSE2b.copy()

In [31]:
datatmp = datatmp.merge(dataUSEstd[["经销商代码", "客户来源更新", "车型new", "车型占比"]], on=["经销商代码", "客户来源更新"], how='left')

In [32]:
noneSTD = datatmp[datatmp['车型占比'].isna()]

In [33]:
datatmp = datatmp[datatmp['车型占比'].notna()]

In [34]:
noneSTD.drop(['车型new', '车型占比'], axis=1, inplace=True)

In [35]:
noneSTD = noneSTD.merge(dataCITYstd[['城市', '车型new', '车型占比']], on=['城市'])

In [36]:
datatmp = pd.concat([datatmp, noneSTD], ignore_index=True)

In [37]:
# 预计算潜客1 是未四舍五入的值， 预计算潜客2 是四舍五入后的值
datatmp['预计算潜客'] = datatmp["潜客量"] * datatmp["车型占比"]
datatmp['预计算潜客1'] = datatmp['预计算潜客'].map(int)
datatmp['预计算潜客2'] = datatmp['预计算潜客'].apply(lambda x: int(x+0.5))
datatmp['进位可能'] = datatmp['预计算潜客'] - datatmp['预计算潜客1']

In [38]:
datatmp.head(20)
datatmp['预计算潜客'].sum()

66503.0

In [ ]:
# 这里计算出来的 差值 就是四舍五入后的差异，需要用这个差异去修改 datatmp
# 里的 预计算潜客2 中不合适的部分。
# 并在 datatmp 中插入新的列 调整潜客
# 所以需要利用 进位可能，进行多退少补：
#     如果需要退n个（差值为负），则寻找 进位可能 > 0.5的数据里最小的n个各去掉 1
#     如果需要补n个（差值为正），则寻找 进位可能 < 0.5的数据里最大的n个各增加 1
# 这部分需要使用函数计算，感觉还比较麻烦
# 这里直接做一个 进位可能 相关的函数，主要是插入 顺序 字段，值是序数：
#   大于 0.5 的 升序，比如：
#     进位可能: [0.51, 0.66, 0.501, 0.7]
#        顺序: [1, 2, 0, 3]
#   小于 0.5 的 降序
##########################################

In [39]:
# 1. 给 datatemp 增加一列，区分 0.5
datatmp['已进位'] = datatmp['进位可能'] >= 0.5

In [40]:
# 都使用的升序，处理时要注意
# 先处理 已进位 为 True的
datatmp['顺序'] = 0
l = datatmp['顺序'] + datatmp[datatmp['已进位']==True].groupby(['经销商代码', '客户来源更新', '已进位'])['进位可能'].rank(ascending=True, method="first").astype('int')
r = datatmp['顺序'] + datatmp[datatmp['已进位']==False].groupby(['经销商代码', '客户来源更新', '已进位'])['进位可能'].rank(ascending=False, method="first").astype('int')
datatmp['顺序'] = l.fillna(0) + r.fillna(0)

In [41]:
# 计算经销商分来源的预计算潜客2的汇总
dataUSEpre = datatmp.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新'])['预计算潜客2'].sum().reset_index()

In [42]:
#dataUSEadj = datatmp.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新'])['预计算潜客2'].sum().reset_index()
#dataUSEadj = dataUSE2b.head(200).merge(dataUSEpre[['经销商代码', '客户来源更新', '预计算潜客2']], on=['经销商代码', '客户来源更新'])
dataUSEadj = dataUSE2b.merge(dataUSEpre[['经销商代码', '客户来源更新', '预计算潜客2']], on=['经销商代码', '客户来源更新'])
dataUSEadj['差值'] = dataUSEadj['潜客量'] - dataUSEadj['预计算潜客2']

In [43]:
dataUSEadj['潜客量'].sum()

66503

In [44]:
# 将 差值 合并进 datatmp
datatmp = datatmp.merge(dataUSEadj[['经销商代码', '客户来源更新', '差值']], on=['经销商代码', '客户来源更新'])

In [45]:
datatmp['预计算真值'] = datatmp.apply(balanceCom, axis=1)

In [46]:
datatmp['预计算真值'].sum()

66503

In [48]:
# 这里开始合并模拟结果和真实结果

In [47]:
dataCOMPUTed = datatmp[['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新','车型new', '预计算真值']].copy()

In [48]:
dataCOMPUTed.rename(columns={'预计算真值': '潜客量'}, inplace=True)
dataCOMPUTed

,大区名称,小区名称,城市,经销商代码,客户来源更新,车型new,潜客量
0,东北区,吉林小区,兴安盟,SA15016,展厅,A3,1
1,东北区,吉林小区,兴安盟,SA15016,展厅,A4 FBU,1
2,东北区,吉林小区,兴安盟,SA15016,展厅,A4L,3
3,东北区,吉林小区,兴安盟,SA15016,展厅,A6 FBU,3
4,东北区,吉林小区,兴安盟,SA15016,展厅,A6L,6
5,东北区,吉林小区,兴安盟,SA15016,展厅,A7,0
6,东北区,吉林小区,兴安盟,SA15016,展厅,A8L,0
7,东北区,吉林小区,兴安盟,SA15016,展厅,Q2L,1
8,东北区,吉林小区,兴安盟,SA15016,展厅,Q3,1
9,东北区,吉林小区,兴安盟,SA15016,展厅,Q5L,1


In [49]:
dataEsti = dataCOMPUTed.append(dataUSEstd[['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新','车型new', '潜客量']])

In [50]:
dataEsti = dataEsti.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新','车型new'])['潜客量'].sum().reset_index()

In [51]:
# 添加车型站比
dataEsti['dealer汇总'] = dataEsti.groupby(['大区名称', '小区名称', '城市', '经销商代码', '客户来源更新'])['潜客量'].transform('sum')
dataEsti['车型占比'] = dataEsti['潜客量']/dataEsti['dealer汇总']
dataEsti['日均概率'] = dataEsti['潜客量']/monthDays

In [52]:
dataEsti['潜客量'].sum()

381473

In [228]:
#### 以下是之前处理的结果

In [78]:
def summarySTAT():
    totalPotentials = databook['潜客量'].sum()
    # dealPotentialsbyDay = databook.groupby(['经销商代码', '日期'])['潜客量'].sum().reset_index()
    totalDealPotentials = databook.groupby(['经销商代码'])['潜客量'].sum().reset_index()
    totalPotentialsFIX = dataEsti['潜客量'].sum()
    # dealPotentialsbyDayFIX = dataEsti.groupby(['经销商代码', '日期'])['潜客量'].sum().reset_index()
    totalDealPotentialsFIX = dataEsti.groupby(['经销商代码'])['潜客量'].sum().reset_index()
    # dealDiffbyDay = dealPotentialsbyDay.merge(dealPotentialsbyDayFIX, on=['经销商代码', '日期'], suffixes=['_origin', '_fix'])
    dealDiff = totalDealPotentials.merge(totalDealPotentialsFIX, on=['经销商代码'], suffixes=['_origin', '_fix'])
    # dealDiffbyDay['差值'] = dealDiffbyDay['潜客量_origin'] - dealDiffbyDay['潜客量_fix']
    dealDiff['差值'] = dealDiff['潜客量_origin'] - dealDiff['潜客量_fix']
    print(dealDiff['差值'].value_counts())
    return totalPotentials - totalPotentialsFIX, dealDiff['差值'].value_counts()

In [111]:
dealerUnique = dataCOMPUTed[['大区名称', '小区名称', '经销商代码', '城市', '客户来源更新']].drop_duplicates()

In [112]:
dealerUnique = dealerUnique.reset_index(drop=True)[['经销商代码', '客户来源更新']]

In [116]:
dataCOMPUTed[(dataCOMPUTed['经销商代码']=='SA22015')&(dataCOMPUTed['客户来源更新']=='展厅')&(dataCOMPUTed['潜客量']>0)][['车型new', '潜客量']]

,车型new,潜客量
36,A4L,5
37,A6L,5
39,Q2L,1
40,Q3,1
41,Q5L,5
44,others,1


In [105]:
dataBYday[(dataBYday['经销商代码']=='SA22015')&(dataBYday['客户来源更新']=='展厅')]

,日期,客户来源更新,大区名称,小区名称,城市,经销商代码,潜客量
0,2020-10-01,展厅,东北区,吉林小区,吉林,SA22015,2
514,2020-10-02,展厅,东北区,吉林小区,吉林,SA22015,1
1632,2020-10-04,展厅,东北区,吉林小区,吉林,SA22015,2
2161,2020-10-05,展厅,东北区,吉林小区,吉林,SA22015,2
2684,2020-10-06,展厅,东北区,吉林小区,吉林,SA22015,3
3207,2020-10-07,展厅,东北区,吉林小区,吉林,SA22015,1
3740,2020-10-08,展厅,东北区,吉林小区,吉林,SA22015,1
5223,2020-10-11,展厅,东北区,吉林小区,吉林,SA22015,1
5805,2020-10-12,展厅,东北区,吉林小区,吉林,SA22015,1
7703,2020-10-16,展厅,东北区,吉林小区,吉林,SA22015,1


In [57]:
pics = []
for pic in graphLVL:
    pics.extend(batDraw(pic))
html = pb.plot_grid(pics, plot_width=300, return_html=True)

In [58]:
with open("tt.html", 'w') as f:
    f.write(html)

In [53]:
dataUSEstd.head()

,大区名称,小区名称,城市,经销商代码,客户来源更新,车型new,潜客量,dealer汇总,车型占比,日均概率
0,东北区,吉林小区,兴安盟,SA15016,展厅,A3,11,237,0.046414,0.354839
1,东北区,吉林小区,兴安盟,SA15016,展厅,A4 FBU,12,237,0.050633,0.387097
2,东北区,吉林小区,兴安盟,SA15016,展厅,A4L,35,237,0.147679,1.129032
3,东北区,吉林小区,兴安盟,SA15016,展厅,A6 FBU,35,237,0.147679,1.129032
4,东北区,吉林小区,兴安盟,SA15016,展厅,A6L,64,237,0.270042,2.064516


In [54]:
dataEsti.head()

,大区名称,小区名称,城市,经销商代码,客户来源更新,车型new,潜客量,dealer汇总,车型占比,日均概率
0,东北区,吉林小区,兴安盟,SA15016,展厅,A3,12,258,0.046512,0.387097
1,东北区,吉林小区,兴安盟,SA15016,展厅,A4 FBU,13,258,0.050388,0.419355
2,东北区,吉林小区,兴安盟,SA15016,展厅,A4L,38,258,0.147287,1.225806
3,东北区,吉林小区,兴安盟,SA15016,展厅,A6 FBU,38,258,0.147287,1.225806
4,东北区,吉林小区,兴安盟,SA15016,展厅,A6L,70,258,0.271318,2.258065


In [65]:
dataEsti[dataEsti['经销商代码']=='SA44031']

,大区名称,小区名称,城市,经销商代码,客户来源更新,车型new,潜客量,dealer汇总,车型占比,日均概率
9253,南部区,深圳小区,深圳,SA44031,展厅,A3,18,1046,0.017208,0.580645
9254,南部区,深圳小区,深圳,SA44031,展厅,A4 FBU,6,1046,0.005736,0.193548
9255,南部区,深圳小区,深圳,SA44031,展厅,A4L,215,1046,0.205545,6.935484
9256,南部区,深圳小区,深圳,SA44031,展厅,A5,4,1046,0.003824,0.129032
9257,南部区,深圳小区,深圳,SA44031,展厅,A6 FBU,28,1046,0.026769,0.903226
9258,南部区,深圳小区,深圳,SA44031,展厅,A6L,242,1046,0.231358,7.806452
9259,南部区,深圳小区,深圳,SA44031,展厅,A7,13,1046,0.012428,0.419355
9260,南部区,深圳小区,深圳,SA44031,展厅,A8L,20,1046,0.019120,0.645161
9261,南部区,深圳小区,深圳,SA44031,展厅,Q2L,45,1046,0.043021,1.451613
9262,南部区,深圳小区,深圳,SA44031,展厅,Q3,38,1046,0.036329,1.225806


In [66]:
dataUSEstd[dataUSEstd['经销商代码']=='SA44031']

,大区名称,小区名称,城市,经销商代码,客户来源更新,车型new,潜客量,dealer汇总,车型占比,日均概率
9156,南部区,深圳小区,深圳,SA44031,展厅,A3,12,716,0.016760,0.387097
9157,南部区,深圳小区,深圳,SA44031,展厅,A4 FBU,4,716,0.005587,0.129032
9158,南部区,深圳小区,深圳,SA44031,展厅,A4L,147,716,0.205307,4.741935
9159,南部区,深圳小区,深圳,SA44031,展厅,A5,3,716,0.004190,0.096774
9160,南部区,深圳小区,深圳,SA44031,展厅,A6 FBU,19,716,0.026536,0.612903
9161,南部区,深圳小区,深圳,SA44031,展厅,A6L,166,716,0.231844,5.354839
9162,南部区,深圳小区,深圳,SA44031,展厅,A7,9,716,0.012570,0.290323
9163,南部区,深圳小区,深圳,SA44031,展厅,A8L,14,716,0.019553,0.451613
9164,南部区,深圳小区,深圳,SA44031,展厅,Q2L,31,716,0.043296,1.000000
9165,南部区,深圳小区,深圳,SA44031,展厅,Q3,26,716,0.036313,0.838710


In [61]:
# 轮盘赌算法，应该用不着了，直接用估计值去填坑
import random
maxnum = 10000
pDIS = [0.046414, 0.050633, 0.147679, 0.147679, 0.270042, 0.008439, 0.004219, 0.033755, 0.025316, 0.07173, 0.004219, 0.008439, 0.181435]
pDISsum = []
for i in range(len(pDIS)):
    if pDISsum:
        pDISsum.append(pDISsum[-1] + pDIS[i])
    else:
        pDISsum.append(pDIS[i])
result = [0] * len(pDIS)
for i in range(maxnum):
    rNUM = random.random()
    for n, m in enumerate(pDISsum):
        if rNUM < m:
            result[n] += 1
            break
print(list(map(lambda x: '%.3f' % x, pDIS)))      
print(list(map(lambda x: '%.3f' % (x/maxnum), result)))

['0.046', '0.051', '0.148', '0.148', '0.270', '0.008', '0.004', '0.034', '0.025', '0.072', '0.004', '0.008', '0.181']
['0.049', '0.049', '0.153', '0.145', '0.272', '0.008', '0.003', '0.031', '0.023', '0.071', '0.005', '0.009', '0.182']
